# Variaciones Information Value

En esta notebook vemos las distintas variantes de Information Value y cuánto laburo extra demandarían

In [1]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("../data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["candidata"] = analizadas["Palabra Candidata"]

In [2]:
palabras_ya_analizadas = analizadas[analizadas['candidata'].notna()]

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [3]:
import re

df = pd.read_csv("../output/provinces_words.csv", index_col=0)

cant_palabras = [c for c in df.columns if re.match(r'.*ocurrencias$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*usuarios$', c)]

print("columnas de palabras => {}\n".format(cant_palabras))
print("columnas de usuarios => {}".format(cant_personas))

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)
df["cant_palabra"] = df[cant_palabras].sum(axis=1)
df["cant_usuarios"] = df[cant_personas].sum(axis=1)

columnas de palabras => ['buenosaires_ocurrencias', 'catamarca_ocurrencias', 'chaco_ocurrencias', 'chubut_ocurrencias', 'cordoba_ocurrencias', 'corrientes_ocurrencias', 'entrerios_ocurrencias', 'formosa_ocurrencias', 'jujuy_ocurrencias', 'lapampa_ocurrencias', 'larioja_ocurrencias', 'mendoza_ocurrencias', 'misiones_ocurrencias', 'neuquen_ocurrencias', 'rionegro_ocurrencias', 'salta_ocurrencias', 'sanjuan_ocurrencias', 'sanluis_ocurrencias', 'santacruz_ocurrencias', 'santafe_ocurrencias', 'santiago_ocurrencias', 'tierradelfuego_ocurrencias', 'tucuman_ocurrencias']

columnas de usuarios => ['buenosaires_usuarios', 'catamarca_usuarios', 'chaco_usuarios', 'chubut_usuarios', 'cordoba_usuarios', 'corrientes_usuarios', 'entrerios_usuarios', 'formosa_usuarios', 'jujuy_usuarios', 'lapampa_usuarios', 'larioja_usuarios', 'mendoza_usuarios', 'misiones_usuarios', 'neuquen_usuarios', 'rionegro_usuarios', 'salta_usuarios', 'sanjuan_usuarios', 'sanluis_usuarios', 'santacruz_usuarios', 'santafe_usuario

In [4]:
df = df[(df.cant_palabra > 80) & (df.cant_usuarios >= 25) ]

Calculemos el valor de la información de cada palabra.

In [5]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log(23) - df["entropy_palabras"]
df["delta_personas"] = np.log(23) - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]
df["ival_norm"] = (1+df["ival_palabras"]) * (1+df["ival_personas"])

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)


Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [6]:
def calcular_porcentaje_candidatas(palabras):
    subconj_analizado = palabras.index.intersection(palabras_ya_analizadas.index)
    candidatas = (analizadas.loc[subconj_analizado]['candidata'] != '0').sum()
    total = subconj_analizado.shape[0]
    print("Palabras candidatas = {}".format(candidatas))
    print("Total analizadas actualmente = {}".format(total))
    
    return candidatas / total

def analizar_palabras_restantes(n, dataframes):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    
    for nombre, this_df in dataframes.items(): 
        print('#'*40)
        print("Métrica: {}".format(nombre))
        palabras = this_df.iloc[:n]
        
        metrica = calcular_porcentaje_candidatas(palabras)
        restantes = {p for p in palabras.index if p not in palabras_ya_analizadas.index}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("Faltan etiquetar {} palabras".format(len(restantes)))
        print("Porcentaje candidatas = {}".format(metrica))
    print("Faltan etiquetar (sin repeticiones) = {}".format(len(todas_las_palabras)))

In [7]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp,
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 176
Total analizadas actualmente = 787
Faltan etiquetar 213 palabras
Porcentaje candidatas = 0.22363405336721728
########################################
Métrica: Personas
Palabras candidatas = 190
Total analizadas actualmente = 709
Faltan etiquetar 291 palabras
Porcentaje candidatas = 0.2679830747531735
########################################
Métrica: PalPer
Palabras candidatas = 182
Total analizadas actualmente = 753
Faltan etiquetar 247 palabras
Porcentaje candidatas = 0.24169986719787517
Faltan etiquetar (sin repeticiones) = 399
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1240
Total analizadas actualmente = 3056
Faltan etiquetar 1944 palabras
Porcentaje candidatas = 0.40575916230366493
########################################
Métrica: PalPer
Palabras candidatas = 1165
Total analizadas actualmente = 3060
Faltan etiquetar

## Sin usar el log(23)

In [8]:
from contrastes.information_value import shuffled_entropy
np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas
df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [9]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 180
Total analizadas actualmente = 794
Faltan etiquetar 206 palabras
Porcentaje candidatas = 0.22670025188916876
########################################
Métrica: Personas
Palabras candidatas = 213
Total analizadas actualmente = 735
Faltan etiquetar 265 palabras
Porcentaje candidatas = 0.2897959183673469
########################################
Métrica: PalPer
Palabras candidatas = 202
Total analizadas actualmente = 787
Faltan etiquetar 213 palabras
Porcentaje candidatas = 0.25667090216010163
Faltan etiquetar (sin repeticiones) = 387
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1335
Total analizadas actualmente = 3178
Faltan etiquetar 1822 palabras
Porcentaje candidatas = 0.4200755191944619
########################################
Métrica: PalPer
Palabras candidatas = 1249
Total analizadas actualmente = 3165
Faltan etiquetar 

## Usando cálculo con normalización "vieja"


In [10]:
np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)


In [11]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [12]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 256
Total analizadas actualmente = 835
Faltan etiquetar 165 palabras
Porcentaje candidatas = 0.3065868263473054
########################################
Métrica: Personas
Palabras candidatas = 370
Total analizadas actualmente = 905
Faltan etiquetar 95 palabras
Porcentaje candidatas = 0.4088397790055249
########################################
Métrica: PalPer
Palabras candidatas = 354
Total analizadas actualmente = 963
Faltan etiquetar 37 palabras
Porcentaje candidatas = 0.367601246105919
Faltan etiquetar (sin repeticiones) = 246
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1650
Total analizadas actualmente = 3427
Faltan etiquetar 1573 palabras
Porcentaje candidatas = 0.48147067405894367
########################################
Métrica: PalPer
Palabras candidatas = 1609
Total analizadas actualmente = 3488
Faltan etiquetar 1512

## Usando el cálculo viejo CASI idéntico

No usó normalización en las dos primeras métricas

In [13]:

np.random.seed(seed=1)

fn = lambda ws: shuffled_entropy(int(ws), len(cant_palabras))


df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)
df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)

In [14]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log(df.cant_palabra)
df["log_personas"] = np.log(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.log_cantidad * df.delta_palabras
df["ival_personas"] = df.log_personas * df.delta_personas
df["ival"] = df.norm_cantidad * df.norm_personas * df.delta_palabras * df.delta_personas

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [15]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 180
Total analizadas actualmente = 795
Faltan etiquetar 205 palabras
Porcentaje candidatas = 0.22641509433962265
########################################
Métrica: Personas
Palabras candidatas = 213
Total analizadas actualmente = 737
Faltan etiquetar 263 palabras
Porcentaje candidatas = 0.28900949796472186
########################################
Métrica: PalPer
Palabras candidatas = 354
Total analizadas actualmente = 962
Faltan etiquetar 38 palabras
Porcentaje candidatas = 0.367983367983368
Faltan etiquetar (sin repeticiones) = 387
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 1329
Total analizadas actualmente = 3175
Faltan etiquetar 1825 palabras
Porcentaje candidatas = 0.41858267716535436
########################################
Métrica: PalPer
Palabras candidatas = 1612
Total analizadas actualmente = 3487
Faltan etiquetar 1